In [17]:
import pandas as pd
import numpy as np
import csv
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from nltk.stem.porter import PorterStemmer
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from string import punctuation
import seaborn as sns
import pandas as pd
import numpy as np
import nltk
import re
import nltk
import matplotlib.pyplot as plt
from scipy import sparse
# Code source: https://degravek.github.io/project-pages/project1/2017/04/28/New-Notebook/
# Dataset from Chakraborty et al. (https://github.com/bhargaviparanjape/clickbait/tree/master/dataset)

In [25]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.linear_model import  LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import *
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

class SentimentAnalysis():

    def __init__(self):        

        columnNames = ["jsonid", "label", "headline_text", "subject", "speaker", "speaker_job_title", "state_info", "party_affiliation", "barely_true_counts", "false_counts", "half_true_counts", "mostly_true_counts", "pants_on_fire_counts", "context","clean", "sentiment_vector","vader_polarity", "sentiment_score"]
        dataTrain = pd.read_csv('input_data/train_sentiment.csv', sep=',', header=None, names = columnNames)
        dataTest = pd.read_csv('input_data/test_sentiment.csv', sep=',', header=None, names = columnNames)

        #dropping columns
        columnsToRemove = ['jsonid', 'label', 'subject', 'speaker','speaker_job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context', 'sentiment_vector']
        dataTrain = dataTrain.drop(columns=columnsToRemove)
        dataTest = dataTest.drop(columns=columnsToRemove)
        dataTrain = dataTrain.iloc[1:] 
        dataTest = dataTest.iloc[1:]
    
    
        tfidfV = TfidfVectorizer(stop_words='english', min_df=5, max_df=30, use_idf=True, smooth_idf=True, token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b')

        #self.logR_pipeline = Pipeline([
        #        ('LogRCV', tfidfV),
        #        ('LogR_clf',LogisticRegression(solver='liblinear', C=32/100))
        #        ])
        
        self.logR_pipeline = Pipeline([
                ('LogRCV', tfidfV),
                ('LogR_clf',RandomForestClassifier(n_estimators=100, max_depth=8, random_state=0))
                ])
        

        self.logR_pipeline.fit(dataTrain['headline_text'],dataTrain['vader_polarity'])
        predicted_LogR = self.logR_pipeline.predict(dataTest['headline_text'])
        score = metrics.accuracy_score(dataTest['vader_polarity'], predicted_LogR)
        print("Sentiment Analysis Model Trained - accuracy:   %0.6f" % score)
        

    def predict(self, text):
        predicted = self.logR_pipeline.predict([text])
        predicedProb = self.logR_pipeline.predict_proba([text])[:,1]
        return bool(predicted), float(predicedProb)
    
    
sa = SentimentAnalysis()
sa.predict("Says the Annies List political group supports third-trimester abortions on demand.")

Sentiment Analysis Model Trained - accuracy:   0.960784


(True, 0.9811315920552556)

In [14]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.linear_model import  LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import *
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

class SentimentAnalysis():

    def __init__(self):        

        columnNames = ["jsonid", "label", "headline_text", "subject", "speaker", "speaker_job_title", "state_info", "party_affiliation", "barely_true_counts", "false_counts", "half_true_counts", "mostly_true_counts", "pants_on_fire_counts", "context","clean", "sentiment_vector","vader_polarity", "sentiment_score"]
        sentimentTrain = pd.read_csv('input_data/sentiment_train.csv')
        sentimentTest = pd.read_csv('input_data/sentiment_test.csv')
        
        train = pd.read_csv('input_data/train_processed.csv')
        test = pd.read_csv('input_data/test_processed.csv')
        
        train = train.merge(sentimentTrain, on = 'clean', how = 'outer')
        test = test.merge(sentimentTest, on = 'clean', how = 'outer')
        
        train.head()
        test.head()
        
        #dropping columns
        #  = ['jsonid', 'label', 'subject', 'speaker','speaker_job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context', 'sentiment_vector']
        #dataTrain = dataTrain.drop(columns=columnsToRemove)
        #dataTest = dataTest.drop(columns=columnsToRemove)
        
        tfidfV = TfidfVectorizer(stop_words='english', min_df=5, max_df=30, use_idf=True, smooth_idf=True, token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b')

        self.logR_pipeline = Pipeline([
                ('LogRCV', tfidfV),
                ('LogR_clf',LogisticRegression(solver='liblinear', C=32/100))
                ])
        
        self.logR_pipeline = Pipeline([
                ('LogRCV', tfidfV),
                ('LogR_clf',RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0))
                ])

        self.logR_pipeline.fit(train['headline_text'],train['polarity'])
        predicted_LogR = self.logR_pipeline.predict(test['headline_text'])
        score = metrics.accuracy_score(test['polarity'], predicted_LogR)
        print("Sentiment Analysis Model Trained - accuracy:   %0.6f" % score)
        

    def predict(self, text):
        predicted = self.logR_pipeline.predict([text])
        predicedProb = self.logR_pipeline.predict_proba([text])[:,1]
        return bool(predicted), float(predicedProb)
    
    
sa = SentimentAnalysis()
sa.predict("Says the Annies List political group supports third-trimester abortions on demand.")

Sentiment Analysis Model Trained - accuracy:   0.533747


(False, 0.5500138866431381)

In [11]:
sentimentTrain = pd.read_csv('input_data/sentiment_train.csv')
sentimentTest = pd.read_csv('input_data/sentiment_test.csv')

train = pd.read_csv('input_data/train_sentiment.csv')
test = pd.read_csv('input_data/test_sentiment.csv')

train_merge = train.merge(sentimentTrain, on = 'clean', how = 'outer')
test_merge = test.merge(sentimentTest, on = 'clean', how = 'outer')

print(test_merge.shape)
test_merge.head()

(1289, 22)


,Unnamed: 0_x,jsonid,label,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,...,mostlytrueocunts,pantsonfirecounts,context,clean,sentiment_vector,vader_polarity,sentiment_score,Unnamed: 0_y,sentiment,polarity
0,80.0,2854.json,pants-fire,Rebuilding three high schools will benefit 40 ...,education,carole-smith,Portland schools superintendent,Oregon,none,0.0,...,0.0,1.0,a press briefing,rebuild three high school benefit percent port...,"[0.0, 0.75, 0.25, 0.4588]",0.0,0.4588,80,0.1,0
1,124.0,10147.json,false,On an income cap for recipients of the popular...,education,jason-carter,State Senator,Georgia,democrat,1.0,...,1.0,0.0,forum,incom cap recipe popular hope scholarship,"[0.0, 0.412, 0.588, 0.6908]",1.0,0.6908,124,0.2,0
2,168.0,7744.json,mostly-true,Says states mandated tests come from an Englis...,education,george-lavender,State representative,Texas,republican,0.0,...,1.0,0.0,in Texas House of Representatives debate,say state mandat test come english company,"[0.0, 1.0, 0.0, 0.0]",0.0,0.0000,168,0.0,0
3,181.0,5470.json,true,Pasco County schools have graduation rates sub...,education,heather-fiorentino,superintendent of Pasco County's public schools,Florida,republican,0.0,...,0.0,1.0,a press release,pisco county school graduate rate substantia h...,"[0.0, 1.0, 0.0, 0.0]",0.0,0.0000,181,-0.3,-1
4,187.0,8320.json,mostly-true,Today many Florida teachers are at risk of hav...,education,charlie-crist,0,Florida,democrat,15.0,...,19.0,2.0,a Tampa Bay Times op-ed,today mani florida teacher risk pay impact per...,"[0.241, 0.759, 0.0, -0.3612]",0.0,-0.3612,187,-0.3,-1


In [6]:
sentimentTrain.head()

,Unnamed: 0,clean,sentiment,polarity
0,0,say anni list polit group support third trimme...,-0.7,-1
1,1,decline coal start start natur gas took start ...,-0.4,-1
2,2,hillary clinton agre john mccain vote give geo...,0.2,0
3,3,health care reform legis like mandat free sex ...,0.2,0
4,4,econom turnaround start end term,0.1,0
